In [1]:
import s3fs
import numcodecs as ncd
import numpy as np

url = "s3://hrrrzarr/sfc/20200801/20200801_00z_anl.zarr/1000mb/TMP/1000mb/TMP/4.3"
s3 = s3fs.S3FileSystem(anon=True)

def retrieve_data(s3_url):
    with s3.open(s3_url, 'rb') as compressed_data:
        buffer = ncd.blosc.decompress(compressed_data.read())

        dtype = "<f2"
        if ("surface/PRES" in s3_url
            or "mean_sea_level/MSLMA" in s3_url
            or "0C_isotherm/PRES" in s3_url): # Pressures above 1000hPa use a larger data type
            dtype = "<f4"

        chunk = np.frombuffer(buffer, dtype=dtype)
        
        gridpoints_in_chunk = 150*150
        number_hours = len(chunk)//gridpoints_in_chunk

        if number_hours == 1: # analysis data is 2d
            data_array = np.reshape(chunk, (150, 150))
        else: # forecast data is 3d but of varying length in time
            data_array = np.reshape(chunk, (number_hours, 150, 150))

    return data_array

chunk_data = retrieve_data(url)
print(np.mean(chunk_data))

316.2
